In [39]:
from PIL import Image, ImageDraw
from face_recognition import face_landmarks
import numpy as np

In [40]:
# load images
face = Image.open("images/thispersondoesnotexist.jpg")
face_array = np.array(face)
mask = Image.open("images/mustache.png")
mask = mask.crop(mask.getbbox())
# create draw object for face image
draw = ImageDraw.Draw(face)

In [41]:
def find_coeffs(pa, pb):
    """
    find coefficientes (so)
    """
    matrix = []
    for p1, p2 in zip(pa, pb):
        matrix.append([p1[0], p1[1], 1, 0, 0, 0, -p2[0]*p1[0], -p2[0]*p1[1]])
        matrix.append([0, 0, 0, p1[0], p1[1], 1, -p2[1]*p1[0], -p2[1]*p1[1]])

    A = np.matrix(matrix, dtype=np.float)
    B = np.array(pb).reshape(8)

    res = np.dot(np.linalg.inv(A.T * A) * A.T, B)
    return np.array(res).reshape(8)


def transform4point(img, src, dst):
    """

    """
    lx, ly = img.size
    mapping = find_coeffs(dst, src)
    img = img.transform((lx*10, ly*10), Image.PERSPECTIVE, mapping)
    img = img.crop(img.getbbox())
    lx, ly = img.size
    # img = img.thumbnail((lx*3,ly*3))

    # img = img.resize((lx*2,ly*2))#, Image.ANTIALIAS)
    return img


def midPoint(a, b, ratio):
    """
    returns the point in between a and b at a distance of ratio*(b-a)
    :param a: point to start the walk
    :param b: point to walk to
    :return: point between a and b, when you walked :ratio: part of total
    """
    ax, ay = a
    bx, by = b
    dx = (bx-ax)*ratio
    dy = (by-ay)*ratio
    return ax+dx, ay+dy


def quadrilateral(landmark, normalize=True):
    """
    Find convenient quadrilateral from 5 landmark points for transformation of mask
    The ratios are based on a perfect frontal face.
    This function can be used for 5 or 68 landmarkpoint by small and large model of the face_landmarks() function

    :param landmark: landmark points based on small or large model from face_recognition
    :param normalize: if True, the smalles xy values are 0. If xy coordinates are given, the smalles xy values will be those coordinates
    :return: 4 xy tuples with corner values of quadrilateral [top_left, top_right, botton_left, bottom_right]
    """
    if len(landmark) == 3:  # model="small"
        lEyeOut, lEyeIn = landmark["left_eye"]
        rEyeOut, rEyeIn = landmark["right_eye"]
        noseTip = landmark["nose_tip"][0]
    elif len(noseTip) == 9: # model="large"
        lEyeOut = landmark["left_eye"][0]
        lEyeIn = landmark["left_eye"][3]
        rEyeOut = landmark["right_eye"][3]
        rEyeIn = landmark["right_eye"][0]
        noseTip = landmark["nose_tip"][2]
    else:
        raise ValueError("Doesn't recognize landmarks; len(nose_tip) should be 1 or 5")
    topLeft     = midPoint(a=lEyeOut, b=lEyeIn,  ratio=0.31)
    topRight    = midPoint(a=rEyeOut, b=rEyeIn,  ratio=0.31)
    bottomLeft  = midPoint(a=lEyeOut, b=noseTip, ratio=0.18)
    bottomRight = midPoint(a=rEyeOut, b=noseTip, ratio=0.18)
    quad = [topLeft, topRight, bottomLeft, bottomRight]
    
    # normalize quadrilateral to (0,0) or give xy tuple/list
    if normalize==True:
        xmin, ymin = [min(a) for a in zip(*quad)]
        quad = [(x-xmin, y-ymin) for x, y in quad]
    elif type(normalize) in [tuple, list]:
        if len(normalize) == 2:
            xmin, ymin = [min(a) for a in zip(*quad)]
            xmin, ymin = xmin+normalize[0], ymin+normalize[1]
            quad = [(x-xmin, y-ymin) for x, y in quad]

    return quad

In [42]:
lx, ly = mask.size
src = [(0,0), (lx,0), (0,ly), (lx,ly)]

In [43]:
landmarks = face_landmarks(face_array, model="small")
for landmark in landmarks:
    dst = quadrilateral(landmark) # TODO add large model
    transMask = transform4point(img=mask, src=src, dst=dst)
    nose = landmark["nose_tip"][0] # TODO add large model
    face.paste(transMask, (nose[0] - transMask.size[0]//2, nose[1]), transMask)
    

In [44]:
face.show()

In [45]:
transMask.show()

In [46]:
def refImage(PILimage, x, y):
    imgArray = np.array(PILimage)        # convert to numpy array
    print(imgArray.shape) 
    refArray = np.zeros(imgArray.shape)[:,:,0] # create black image array
    refArray[y,x] = 255                  # with a white pixel at the reference point
    return Image.fromarray(refArray)     # convert to PIL image object

refmask = refImage(mask, x=250, y=100)
refmask.show()

(107, 456, 4)


In [47]:
maks = Image.open("images/mustache.png")
refmask = refImage(mask, x=250, y=100)
transMask = transform4point(img=mask, src=src, dst=dst)
transMask.show()

(107, 456, 4)


In [48]:
type((1,2))

tuple